# WP 0: Verständnis

## Weisenfeiler-Lehman Kernels

### Klassifikation:

- Ziel: Einer chemischen Reaktion wird eine Klasse zugeordnet (z. B. Oxidation, Reduktion).
- Wir haben gelabelte Daten (Reaktionen mit Klassen). Ein Modell soll aus Merkmalen lernen, diese Klassen vorherzusagen.

### SVMs:
Support Vector Machines (SVMs) arbeiten oft mit Kerneln. Ein Kernel misst “Ähnlichkeit” zwischen zwei Objekten, ohne dass wir explizit alle Merkmale als Vektoren vorhalten müssen.

- Sei χ eine Menge von Objekten (z. B. Graphen).
- Ein Kernel ist eine Funktion k: χ × χ → ℝ.
- Es gibt eine Merkmalsabbildung Φ: χ → H in einen Merkmalsraum H (Hilbertraum).
- Der Kernel entspricht dem Skalarprodukt im Merkmalsraum:
    - k(x, y) = ⟨Φ(x), Φ(y)⟩



- Statt echte Vektoren zu bauen, repräsentieren wir Merkmale als Hash-Mengen (Sets von Hashes).
- Dann zählt das “Skalarprodukt” einfach, wie viele gemeinsame Elemente beide Mengen haben:
    - k(x, y) = |Φ(x) ∩ Φ(y)|


### Weisfeiler-Lehman (WL) Grundlage: Wie entstehen G0, G1, ..., Gh?

Ein Graph G besteht aus:

- V: Knoten (z. B. Atome)
- E: Kanten (z. B. Bindungen)
- l: Labels (Bezeichnungen der Knoten/Kanten)


WL macht h Iterationen. In jeder Iteration werden die Knotenlabels durch die Nachbarschaftsinformationen “angereichert” und neu gehasht. Die Struktur (V, E) bleibt gleich; nur die Labels ändern sich:


- G0 = (V, E, l0) = ursprünglicher Graph
- G1 = (V, E, l1), G2 = (V, E, l2), ..., Gh = (V, E, lh)
- Nach jeder Iteration “weiß” ein Knoten mehr über seine Nachbarschaft, was die Struktur besser beschreibt.

#### WL-Kernel: Summe über alle Iterationen


- Wir nehmen einen Basis-Kernel k (z. B. Vertex-, Edge- oder Shortest-Path-Kernel) und wenden ihn auf jedes Gi an.
- Dann summieren wir über alle Iterationen:
    - kWL(h)(G, H) = k(G0, H0) + k(G1, H1) + … + k(Gh, Hh)
- Dadurch fließt Information von unmittelbaren Nachbarn (Iteration 0) bis zu weiter entfernten Nachbarschaften (Iteration h) ein.

#### Basis-Kernel 1: Vertex (WL Subtree Kernel)

- Zählt, wie viele Knotenlabels übereinstimmen.
- ΦV(G) gibt die Menge aller Knotenlabels l(a) für a ∈ V(G).
- Definition:
    - kV(x, y) = ⟨ΦV(x), ΦV(y)⟩
    - Für zwei Graphen G, H:
        - kV(G, H) = ∑{u∈V(G)} ∑{v∈V(H)} δ(l(u), l(v))
    - δ ist der Dirac-Kernel:
        - δ(x, y) = 1, wenn x = y
        - δ(x, y) = 0, sonst
- Knoten mit gleichen Labels gelten als “gleichartige Strukturpunkte”.

#### Basis-Kernel 2: Edge (Kanten-Kernel)

- Zählt, wie viele Kanten mit passenden Endknoten- und Kantenlabels übereinstimmen.
- ΦE(G) ist die Menge von Dreifach-Labels l(a) l(ab) l(b) für jede Kante ab ∈ E(G).
- Definition:
    - kE(x, y) = ⟨ΦE(x), ΦE(y)⟩
    - Für G, H:
        - kE(G, H) = ∑{(a,b)=e∈E(G)} ∑{(a′,b′)=e′∈E(H)} δ(l(a), l(a′)) · δ(l(b), l(b′)) · δ(l(e), l(e′))
- Zwei Kanten “matchen”, wenn ihre beiden Endknoten-Labels und das Bindungslabel identisch sind.

#### Basis-Kernel 3: Shortest-Path (Kürzeste-Pfade-Kernel)

- Zählt, wie viele kürzeste Pfade mit gleichen Label-Sequenzen übereinstimmen.
- ΦSP(G) erzeugt für jedes Knotenpaar (a, b) die Sequenz der Labels entlang des kürzesten Pfades von a nach b.
- Definition:
    - kSP(x, y) = ⟨ΦSP(x), ΦSP(y)⟩
    - Praktisch: Vergleiche die Label-Sequenzen aller kürzesten Pfade beider Graphen und zähle die Übereinstimmungen.
    - Die kürzesten Pfade zwischen allen Knotenpaaren kann man z. B. mit dem Floyd-Warshall-Algorithmus berechnen.

#### Praktische Umsetzung mit Hash-Mengen

- Für jeden Graphen und jede Iteration i berechnen wir seine Feature-Menge Φ(Gi) (je nach Basis-Kernel).
- Wir hashen die einzelnen Features und sammeln sie in einem Set:
    - SGi = {Hashes der Features von Gi}
- Der Kernel zwischen zwei Graphen in Iteration i ist die Anzahl gemeinsamer Hashes:
    - k(Gi, Hi) = |SGi ∩ SHi|
- WL-Kernel ist dann die Summe über alle Iterationen:
    - kWL(h)(G, H) = ∑_{i=0}^{h} |SGi ∩ SHi|



## ITS-Graphen (Imaginary Transition State Graphs)

### chemische Reaktionen:

- Chemische Reaktionen verändern Bindungen zwischen Atomen (Bonds), nicht die Atome selbst.
- Atome bleiben erhalten, nur ihre Verbindungen ändern sich: manche Bindungen werden gebrochen, andere neu gebildet.



### Atom-zu-Atom-Karte (AAM)

- Um eine Reaktion präzise zu beschreiben, braucht man eine Zuordnung der Atome von Edukten (Reaktanten) zu Produkten.
- Formal: α: V(G) → V(H) ist eine Bijektion zwischen den Knoten (Atomen) der Eduktgraphen G und Produktgraphen H.
- α bewahrt Atomtypen: Wenn ein Atom in G Kohlenstoff ist, muss sein Bild α(x) in H ebenfalls Kohlenstoff sein.

#### Darstellung als Graphen

- Wir betrachten Strukturformeln als Graphen:
    - G: Graph der Edukte (Atome = Knoten V(G), Bindungen = Kanten E(G))
    - H: Graph der Produkte (Atome = Knoten V(H), Bindungen = Kanten E(H))
    - a: Knotenlabel (Atomtyp), b: Kantenlabel (Bindungstyp, z. B. Einfach-/Doppelbindung)
- Atomabbildung α verknüpft Knoten aus G mit Knoten aus H.

#### Hilfsgraph Γ(G, H, α)

- Ziel: Alles zusammen in einem einzigen Graphen sichtbar machen (Edukte, Produkte und die Atomzuordnung).
- Definition von Γ:
    - Knoten: V(Γ) = V(G) ⊍ V(H) (disjunkte Vereinigung beider Knotenmengen)
    - Kanten: E(Γ) = E(G) ⊍ E(H) ⊍ E(α), wobei E(α) = { x α(x) | x ∈ V(G) } die Zuordnungskanten sind
    - Knotenlabels:
        - Für x ∈ V(G): (a_G(x), 1)
        - Für x ∈ V(H): (a_H(x), 2)
        - Die zweite Komponente (1 oder 2) markiert, ob der Knoten zu G oder H gehört
    - Kantenlabels:
        - Für e ∈ E(G): b(e) = b_G(e)
        - Für e ∈ E(H): b(e) = b_H(e)
        - Für e ∈ E(α): b(e) = * (Sonderlabel für Zuordnungskanten)
    - Γ zeigt beide Molekülgraphen und die 1:1-Atomzuordnung als zusätzliche, “dotted” (gedachte) Kanten.

### ITS-Graph Υ(G, H, α)

- Der ITS-Graph ist die komprimierte Darstellung des “gedachten” Übergangszustands:
- Definition von Υ:
    - Knoten: V(Υ) = V(G) (wir bleiben auf der Edukt-Knotenmenge)
    - Knotenlabels: a_Υ(x) = a_G(x) (Atomtypen bleiben wie in G)
    - Kanten: xy ∈ E(Υ), genau dann wenn:
        - xy ∈ E(G) (Bindung existiert in Edukten) oder
        - α(x)α(y) ∈ E(H) (entsprechende Bindung existiert in Produkten)
    - Kantenlabels als Tupel:
        - b(xy) =
            - (b_G(xy), b_H(α(x)α(y))) falls Bindung in beiden Seiten existiert
            - (b_G(xy), ) falls nur in G vorhanden (Bindung wird gebrochen)
            - (, b_H(α(x)α(y))) falls nur in H vorhanden (Bindung wird gebildet)
    - Intuition:
        - Eine ITS-Kante existiert, wenn die Bindung mindestens auf einer Seite (G oder H) existiert.
        - Das Kantenlabel ist ein Paar, das den Bindungstyp “vorher/nachher” zeigt:
            - Gleiches Label beidseits: unverändert
            - Unterschiedliche Labels: geändert (z. B. Doppelbindung → Einfachbindung)
            - Nur links: gebrochen
            - Nur rechts: neu gebildet

    - Wichtiger mathematischer Befund

    - Zwei Atomzuordnungen α und β sind genau dann identisch, wenn ihre
    - ITS-Graphen isomorph sind:
        - α = β ⇔ Υ(G, H, α) ≅ Υ(G, H, β)
    - Das heißt: Der ITS-Graph kodiert die Zuordnung vollständig und eindeutig.

#### Reaktionszentrum

- Das Reaktionszentrum ist die Menge der geänderten Bindungen und die direkt angrenzenden Atome.
- Im ITS-Graphen ist das genau der Teilgraph, der durch Kanten mit “divergenten” Labeln induziert wird:
    - Also dort, wo das Tupel (b_G, b_H) nicht gleich ist (z. B. (=/-, -/*), etc.)
    - Intuition: Das Reaktionszentrum zeigt, wo in der Struktur tatsächlich die Änderungen stattfinden.

- Warum ITS-Graphen nützlich sind

- Sie verknüpfen die Struktur von Edukten und Produkten plus die explizite Atomzuordnung in einem Graphen.
- Damit kann man Ähnlichkeiten zwischen Reaktionen als Graphähnlichkeiten messen.
    Im Praktikum:
        Wir wenden Weisfeiler-Lehman Graph-Kernel auf ITS-Graphen an, um Reaktionen zu klassifizieren.
        Vorteil: Der ITS-Graph macht das “Mechanismus-Muster” (gebrochen/gebildet/geändert) explizit sichtbar, was für das Lernen hilfreich ist.




## DRF (Differential Reaction Fingerprint)

- ITS-Graphen brauchen eine genaue Atom-zu-Atom-Zuordnung (AAM). In der Praxis ist diese oft unbekannt oder ungenau.
- DRF umgeht dieses Problem: Es beschreibt die Reaktion, ohne eine explizite AAM zu benötigen.

- Grundidee: Reaktionen = Änderungen in Bindungen und Nachbarschaften

- Wie beim ITS gilt: Eine Reaktion ist durch veränderte Bindungen und damit veränderte Nachbarschaften definiert.
- Statt die Atome explizit zuzuordnen, schauen wir getrennt auf die Strukturmerkmale der Edukte und Produkte.

#### Feature-Transformation Φ

- Φ ist eine Abbildung, die einen chemischen Graphen in eine Feature-Menge überführt (kein Vektor, sondern Set).
- Beispiele für Φ (wie bei WL-Kerneln):
    - Vertex-Features: Knotenlabels
    - Edge-Features: Kantenlabels inklusive Endknoten
    - Shortest-Path-Features: Label-Sequenzen entlang kürzester Pfade
    - Praktisch werden diese Features gehasht und als Menge gespeichert.

- DRF-Definition: Symmetrische Differenz der Feature-Mengen

- Wir berechnen Φ separat für Edukte (ER) und Produkte (PR).
    - Features, die in beiden Mengen vorkommen, gehören zu den unveränderten
    - Bereichen der Moleküle und sind für die Reaktionsbeschreibung uninteressant.
    - Daher definieren wir den Fingerprint der Reaktion als die symmetrische
    
- Differenz:
    - Φreaction(R = (ER, PR)) := Φ(ER) ∆ Φ(PR)
    - Das ist äquivalent zu:
        - Φreaction(R) = (Φ(ER) \ Φ(PR)) ∪ (Φ(PR) \ Φ(ER))
    - Intuition:
        - Feature nur in ER → beschreibt “vorher vorhanden, nachher nicht” (z. B. gebrochene Bindung oder verlorene Nachbarschaft).
        - Feature nur in PR → “vorher nicht, nachher vorhanden” (z. B. neu gebildete Bindung oder entstandene Nachbarschaft).
        - Feature in beiden → unverändert, daher nicht Teil des Fingerprints.

    Gleiche Basis-Φ wie bei ITS

    Wichtig: Das grundlegende Φ ist dasselbe wie bei ITS-Graphen (Vertex/Edge/Shortest-Path mit WL-Iterationen).
    Unterschied: DRF vergleicht die Feature-Sets der Edukte und Produkte direkt, ohne AAM.

    Umgang mit Reaktionszeichnungen und Reagenzien

    DRF unterscheidet nicht explizit zwischen “Reaktanten” und “Reagenzien” (die Grenze ist oft unklar).
    In der Praxis werden Reagenzien dem Eduktteil zugeschlagen; entscheidend ist die symmetrische Differenz der Strukturmerkmale, nicht die strikte Trennung.

    Hashen und optionales Hash-Folding

    Aus den Features wird ein Hash-Set gebildet (zur Speicher- und Geschwindigkeitsoptimierung).
    Optional: Hash-Folding (Reduktion der Hashlänge), um die Menge kompakter zu machen. Es ist ein Kompromiss zwischen Kollisionen und Speicherbedarf.

    Anwendung im Klassifikationsworkflow

    Für jede Reaktion berechnest du:
        Für jede WL-Iteration i die Feature-Sets der Edukte Φi(ER) und Produkte Φi(PR).
        Deren symmetrische Differenz Si(R) = Φi(ER) ∆ Φi(PR).
    Der endgültige DRF-Fingerprint ist die Vereinigung über alle Iterationen:
        S(R) = ⋃_{i=0}^{h} Si(R)
    Für den Kernel zwischen zwei Reaktionen R und Q (nur über Sets, ohne Vektoren):
        kDRF(R, Q) = |S(R) ∩ S(Q)|
    Dieser Kernel kann direkt mit einer SVM verwendet werden (ähnlich wie bei WL-Kerneln auf ITS).
